In [21]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize

In [48]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/10_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_10ind = df.iloc[:1171].copy()
df_10ind['Date'] = pd.to_datetime(df_10ind['Date'], format='%Y%m')
df_10ind.set_index('Date', inplace=True)
df_10ind = df_10ind.apply(pd.to_numeric, errors='coerce')
df_10ind = df_10ind / 100

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')

df_marketCap = df_numfirm * df_avgsize

In [45]:
df_10ind

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1926-07-01,0.0145,0.1555,0.0469,-0.0118,0.0290,0.0083,0.0011,0.0177,0.0704,0.0213
1926-08-01,0.0397,0.0368,0.0281,0.0347,0.0266,0.0217,-0.0071,0.0425,-0.0169,0.0435
1926-09-01,0.0114,0.0480,0.0115,-0.0339,-0.0038,0.0241,0.0021,0.0069,0.0204,0.0029
1926-10-01,-0.0124,-0.0823,-0.0363,-0.0078,-0.0458,-0.0011,-0.0229,-0.0057,-0.0263,-0.0284
1926-11-01,0.0520,-0.0019,0.0410,0.0001,0.0471,0.0163,0.0643,0.0542,0.0371,0.0211
...,...,...,...,...,...,...,...,...,...,...
2023-09-01,-0.0457,-0.0258,-0.0708,0.0317,-0.0598,-0.0322,-0.0568,-0.0471,-0.0504,-0.0341
2023-10-01,-0.0353,-0.1788,-0.0276,-0.0624,-0.0178,-0.0018,0.0047,-0.0458,0.0112,-0.0253
2023-11-01,0.0502,0.1576,0.0868,-0.0129,0.1196,0.0697,0.0718,0.0587,0.0508,0.1041


In [60]:
def rolling_pf_SR(returns, rf, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def negativeSR(w):
            R = np.sum(z_bar * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            SR = (R - rf) / V
            return -SR

        initial_weights = np.ones(num_assets) / num_assets
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(negativeSR, initial_weights, method='SLSQP', constraints=constraints)
        pf_weights.iloc[i] = result.x
        
    df = pf_weights.dropna()
        
    returns = df.loc[df.index]

    next_month_returns = returns.shift(-1)    

        
    df["Returns"] = (df * next_month_returns).sum(axis=1)


    return df.dropna()

In [61]:
rolling_pf_SR(df_10ind, 0.03, 60)

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1838/2668288717.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Returns"] = (df * next_month_returns).sum(axis=1)


,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Returns
Date,,,,,,,,,,,
1931-07-01,-7688.790073,7571.093137,-1191.548228,-33591.106252,24772.894318,8252.161218,-13484.765434,6825.506179,28197.235799,-19661.680664,935816427.331187
1931-08-01,-2273.355611,1353.623869,-894.205964,-9101.199858,7209.398414,2312.955991,-2992.027443,2778.166392,7854.733954,-6247.089745,290673785.537763
1931-09-01,-2963.824344,788.027843,-1168.871367,-8898.583592,8376.874173,2602.819624,-2457.172593,2718.332933,8919.998025,-7916.600702,287981893.729449
1931-10-01,-3007.751285,468.704587,-644.49633,-7752.356349,8240.663012,1958.127972,-2153.436432,1796.509732,7918.83211,-6823.797018,338230401.046692
1931-11-01,-4133.016692,1716.62737,-1583.781545,-9304.693404,10759.318491,1703.734672,-1970.521463,2762.996197,10240.844861,-10190.50849,475659595.042543
...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,-4816.649316,22489.965292,-2290.816229,4553.440797,7155.779166,-12666.178985,-1454.85528,-4180.491049,-4811.532626,-3977.661771,212831614.252351
2023-10-01,-1223.337736,5849.121131,-840.857439,1367.923753,1699.797506,-3208.516569,-494.106863,-1165.168144,-1143.735896,-840.119743,91620623.650581
2023-11-01,-2987.529438,8941.42759,-915.432468,2846.42006,3563.99148,-5892.728306,17.042943,-1986.991007,-2368.791728,-1216.409127,181958891.976369


In [40]:
def calculate_portfolio_returns_next_month(portfolio_weights, portfolio_returns):
    # Assurez-vous que les index des deux DataFrames correspondent
    portfolio_returns = portfolio_weights.loc[portfolio_weights.index]

    # Décaler les rendements d'un mois
    next_month_returns = portfolio_returns.shift(-1)

    # Multipliez les poids par les rendements décalés et sommez-les pour obtenir le rendement total du portefeuille
    total_portfolio_returns_next_month = (portfolio_weights * next_month_returns).sum(axis=1)

    return total_portfolio_returns_next_month

# Utilisez la fonction pour calculer les rendements totaux du portefeuille avec les rendements du mois suivant
total_returns_next_month = calculate_portfolio_returns_next_month(rolling_pf_SR(df_10ind, 0.03, 60), df_10ind)


In [42]:
total_returns_next_month/100

Date
1931-07-01         0.208644
1931-08-01          0.21937
1931-09-01       146.062786
1931-10-01    193701.194001
1931-11-01    137691.810896
                  ...      
2023-09-01         0.387523
2023-10-01         0.587983
2023-11-01         1.233622
2023-12-01         0.573669
2024-01-01              0.0
Length: 1111, dtype: object

In [57]:
def rolling_pf_SR_NS(returns, rf, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def negativeSR(w):
            R = np.sum(z_bar * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            SR = (R - rf) / V
            return -SR

        initial_weights = np.ones(num_assets) / num_assets
        bounds = [(0, None) for _ in range(num_assets)]
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(negativeSR, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()

In [58]:
rolling_pf_SR_NS(df_10ind, 0.03, 60)

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1931-07-01,0.0,0.0,0.0,0.0,0.546355,0.0,0.0,0.0,0.453645,0.0
1931-08-01,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1931-09-01,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1931-10-01,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1931-11-01,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-10-01,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-11-01,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
def rolling_pf_VAR(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        # Calculer les poids en utilisant la formule
        var = window.var()  # Calcule l'écart-type de chaque actif sur la fenêtre
        weights = (1 / var) / np.sum(1 / var)  # Calculer les poids en utilisant la formule
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()

In [27]:
def rolling_pf_VOL(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        # Calculer les poids en utilisant la formule
        std = window.std()  # Calcule l'écart-type de chaque actif sur la fenêtre
        weights = (1 / std) / np.sum(1 / std)  # Calculer les poids en utilisant la formule
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()

In [28]:
def rolling_pf_SameWeights(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        weights = (1 / window.shape[1])
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()

In [29]:
def rolling_pf_MarketCap(returns, window_size):

    pf_weights = pd.DataFrame(index=df_10ind.index, columns=df_10ind.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        weights = window / window.sum()
        
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()  

In [30]:
#rolling_pf_MarketCap(df_10ind, 60)

In [31]:
def rolling_pf_minVAR(returns, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def min_volatility(w):
            w = np.array(w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            return V

        initial_weights = np.ones(num_assets) / num_assets
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(min_volatility, initial_weights, method='SLSQP', constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()